### Load QM9 model and predict

In [1]:
from megnet.models import MEGNetModel
import numpy as np
from operator import itemgetter
import json

def get_graph_from_doc(doc):
    """
    Convert a json document into a megnet graph
    """
    atom = [i['atomic_num'] for i in doc['atoms']]

    index1_temp = [i['a_idx'] for i in doc['atom_pairs']]
    index2_temp = [i['b_idx'] for i in doc['atom_pairs']]
    bond_temp = [i['spatial_distance'] for i in doc['atom_pairs']]

    index1 = index1_temp + index2_temp
    index2 = index2_temp + index1_temp
    bond = bond_temp + bond_temp
    sort_key = np.argsort(index1)
    it = itemgetter(*sort_key)

    index1 = it(index1)
    index2 = it(index2)
    bond = it(bond)
    graph = {'atom': atom, 'bond': bond, 'index1': index1, 'index2': index2, 'state': [[0, 0]]}
    return graph
    
# load an example qm9 document
with open('../megnet/data/tests/qm9/000001.json', 'r') as f:
    doc = json.load(f)
# convert to a graph
graph = get_graph_from_doc(doc)

In [2]:
graph

{'atom': [6, 1, 1, 1, 1],
 'bond': (1.0919181,
  1.0919425,
  1.0918945,
  1.0919342,
  1.7830887,
  1.783101,
  1.7831048,
  1.0919181,
  1.7831084,
  1.7831007,
  1.0919425,
  1.7830887,
  1.7831084,
  1.783101,
  1.7831069,
  1.0918945,
  1.7831048,
  1.7831007,
  1.0919342,
  1.7831069),
 'index1': (0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4),
 'index2': (1, 2, 3, 4, 2, 3, 4, 0, 3, 4, 0, 1, 2, 1, 4, 0, 1, 2, 0, 3),
 'state': [[0, 0]]}

In [2]:
# all target names
names = ['mu', 'alpha', 'HOMO', 'LUMO', 'gap', 'R2', 'ZPVE', 'U0', 'U', 'H', 'G', 'Cv', 'omega1']


y_pred = []
y_true = []

print('*** Result Comparisons ***')
print('Target\tMEGNet\tQM9')

for i in names:
    model = MEGNetModel.from_file('../mvl_models/qm9-2018.6.1/' + i+'.hdf5')
    pred = model.predict_graph(graph)
    y_pred.append(pred)
    y_true.append(doc['mol_info'][i])
    print('%s\t%.3f\t%.3f' %(i, y_pred[-1], float(y_true[-1])))


*** Result Comparisons ***
Target	MEGNet	QM9
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


mu	-0.008	0.000
alpha	13.127	13.210
HOMO	-10.557	-10.550
LUMO	3.241	3.186
gap	13.622	13.736
R2	35.975	35.364
ZPVE	1.215	1.218
U0	-17.166	-17.172
U	-17.353	-17.286
H	-17.420	-17.389
G	-16.107	-16.152
Cv	6.427	6.469
omega1	3151.626	3151.708


In [6]:
index1=[]
index2=[]
i=0
atom_num=len(names['dic9']["Z"])
for i in range(atom_num):
    j=0
    while j < atom_num:
        if i != j:
            index1.append(i)
            index2.append(j)
        j+=1
index1=tuple(index1)
index2=tuple(index2)

len_neibor=len(index1)
bonds=[]
for i in range(len_neibor):
    bonds.append(spatial_dis(names['dic9']["Coordinate"][index1[i]],names['dic9']["Coordinate"][index2[i]]))
bonds=tuple(map(float,bonds))

In [7]:
graph={"atom":names["dic9"]["Z"],'bond':bonds,'index1':index1,"index2":index2,'state': [[0, 0]]}



In [13]:
from re import U
import pandas as pd
import numpy as np
import time
from glob import glob
import torch
def spatial_dis(a,b):
    return torch.norm(torch.tensor(a)-torch.tensor(b))

neighborlist = {}
for _ in range(11,30,1):
    neighborlist[str(_)]={}
    neighborlist[str(_)]["index1"]=[]
    neighborlist[str(_)]["index2"]=[]
    i=0
    for i in range(_):
        j=0
        while j < _:
            if i != j:
                neighborlist[str(_)]["index1"].append(i)
                neighborlist[str(_)]["index2"].append(j)
            j+=1
    neighborlist[str(_)]["index1"]=tuple(neighborlist[str(_)]["index1"])
    neighborlist[str(_)]["index2"]=tuple(neighborlist[str(_)]["index2"])

graph_all={}
id_list=[]
path='/work/gp23/p23002/AI4Science_ICML/guyuhan/Alchemy/pyg/data-bin/raw/alchemy_data/Alchemy-v20191129/final_version.csv'
df=pd.read_csv(path)                        #读入所有数据
start=time.time()
col_name = df.columns.values                #column name
sort1 = col_name[1]                         #原子数目
sort2 = col_name[0]                         #‘gdb_idx’
df = df.sort_values(by=[sort1,sort2])       #对原子数以及序号进行排序
data={}
data["xyz"]={}
data["atom"]={}
names=locals()
#names 中包含 df9～12 dic9～12 以及 dic 9～12 * （14+2）
# for i in [9,10,11,12]:
#这是一个循环用于处理整个的Alchemy中的重原子数为9～12的数据，如果只处理9的可以注释掉并开启下一行，并且下面的也要同步注释
for i in [9]:
    names['df'+str(i)]=df[df['atom number'].isin([i])]
    names['dic'+str(i)]={}
    #新定义的df9～12 以及 dic9～12
    names['dic'+str(i)]['Z']=[]#Z是原子序数
    names['dic'+str(i)]['Coordinate']=[]#坐标
    names['dic'+str(i)]["Contents"]=[]
    #Z 和 Coordinate 是很多维的数据，需要从XYZ文件中读取
    #其他的一些属性是从csv文件中直接读取的
    # for j in range(len(df.columns.values)):
    #     names['dic'+str(i)][col_name[j].split('\n')[0]]=[]

search_table = {"H": 1, "B": 5, "C": 6, "N": 7, "O": 8, "F": 9, "Si": 14, "P": 15, "S": 16, "Cl": 17, "Se": 34, "Br": 35}
# Z 中的转换表哥
# direc='/work/gp23/p23002/AI4Science_ICML/guyuhan/Alchemy/pyg/data-bin/raw/alchemy_data/Alchemy-v20191129/atom_'
# #这是xyz文件的地址
# def sort_key(elem):
#     return int(elem.replace(direc,'').replace('.xyz','').split('/')[1])
# ev=27.211386245988
# delta_U0_ref={'H':-0.500273,'C':-37.846772,'N':-54.583861,'O':-75.064579,'F':-99.718730}
# delta_U_ref={'H':-0.498857,'C':-37.845355,'N':-54.582445,'O':-75.063163,'F':-99.717314}
# delta_H_ref={'H':-0.497912,'C':-37.844411,'N':-54.581501,'O':-75.062219,'F':-99.716370}
# delta_G_ref={'H':-0.510927,'C':-37.861317,'N':-54.598897,'O':-75.079532,'F':-99.733544}
# for i in [9,10,11,12]:
for i in [9]:
    # names['xyz_filepath_list'+str(i)] = list(glob('/work/gp23/p23002/AI4Science_ICML/guyuhan/Alchemy/pyg/data-bin/raw/alchemy_data/Alchemy-v20191129/atom_'
    #                                               +str(i)+'/*.xyz'))
    # names['xyz_filepath_list'+str(i)].sort(key=sort_key)
    names['delta_dic'+str(i)]={}
    # preperty=['delta_U0','delta_U','delta_H','delta_G']
    # for j in range(len(preperty)):
    #     names['delta_dic'+str(i)][preperty[j]] = []
    for k in range(len(names['df'+str(i)][col_name[0]])):
        names['dic'+str(i)]['Z']=[]
        names['dic'+str(i)]["Coordinate"]=[]
    #下面这一句是用来测试的
    # for k in range(1):
        if k == 1000:
            break
        id = names['df'+str(i)][col_name[0]].iloc[k]
        flag=True
        with open('/work/gp23/p23002/AI4Science_ICML/guyuhan/Alchemy/pyg/data-bin/raw/alchemy_data/Alchemy-v20191129/atom_'
                  +str(i)+'/'+str(id) + ".xyz") as f:
            contents = f.readlines()
            
            for lineID in range(2,len(contents)):
                if contents[lineID].strip().split()[0] =='S' or contents[lineID].strip().split()[0] =='Cl':
                    flag=False
        if flag == True:
            id_list.append(id)
            with open('/work/gp23/p23002/AI4Science_ICML/guyuhan/Alchemy/pyg/data-bin/raw/alchemy_data/Alchemy-v20191129/atom_'
                  +str(i)+'/'+str(id) + ".xyz") as f:
                contents = f.readlines()
                # tempU0=0
                # tempU=0
                # tempH=0
                # tempG=0
                for lineID in range(2, len(contents)):
                    names['dic'+str(i)]["Z"].append(search_table[contents[lineID].strip().split()[0]])
                    names['dic'+str(i)]["Coordinate"].append(list(map(float, contents[lineID].strip().split()[1: ])))
                
            atom_number=(contents[0].strip())
            index1=neighborlist[str(atom_number)]['index1']
            index2=neighborlist[str(atom_number)]['index2']
            bonds=[]
            for m in range(len(index1)):
                bonds.append(spatial_dis(names['dic'+str(i)]["Coordinate"][index1[m]],names['dic'+str(i)]["Coordinate"][index2[m]]))
            bonds=tuple(map(float,bonds))
            graph={"atom":names["dic"+str(9)]["Z"],'bond':bonds,'index1':index1,"index2":index2,'state': [[0, 0]]}
            graph_all[str(id)]=graph
                    

In [ ]:
targets = ['mu','HOMO']
names=locals()


y_pred = []
y_true = []

print('*** Result Comparisons ***')
# print('Target\tMEGNet\tQM9')

for target in targets:
    names[target+"pred"]=[]
    model = MEGNetModel.from_file('../mvl_models/qm9-2018.6.1/' + i+'.hdf5')
    for id in id_list:
        pred = model.predict_graph(graph_all[str(id)])
        names[target+"pred"].append(pred)


[array([1.5266045], dtype=float32), array([1.1837761], dtype=float32)]

### Predict from SMILES

In [3]:
from megnet.utils.molecule import get_pmg_mol_from_smiles

MODEL_NAME = 'HOMO'

model = MEGNetModel.from_file('../mvl_models/qm9-2018.6.1/%s.hdf5' % MODEL_NAME)

In [4]:
# The smiles of qm9:000001 is just C
mol1 = get_pmg_mol_from_smiles('C')
model.predict_structure(mol1)

array([-10.557696], dtype=float32)

The result matches with previous results when we compute it from pre-computed graph